In [5]:
from nba_py import player
from nba_py.player import get_player
from nba_py import constants
from nba_py import team
from nba_py import game
from nba_py.constants import TEAMS

import pickle
import re
import time
import json
import pprint as pp
import sys
import pandas as pd

from time import time,sleep

In [7]:
def scrape_matchups(): 
    
    df = pd.read_pickle('../data/player_game_table.pkl')
    seasons = ["2017-2018", "2016-2017", "2015-2016"]
    #print(df.head(1))
    game_ids = df.loc[(df.player_id == "2544") & (df.season.isin(seasons))]
    player_ids = set([])
    count = 0
    for idx, row in game_ids.iterrows():
        if not count%10:
            print(count, " out of ", game_ids.shape)
        count += 1
        
        all_games = df.loc[(df.game_id == row.game_id) & (df.team_id != row.team_id)]
        #print(all_games)
        for idx, game_row in all_games.iterrows():
            player_ids.add(game_row.player_id)

    print(len(player_ids))
    print(player_ids)
    
    return player_ids

In [8]:
player_ids = scrape_matchups()

0  out of  (232, 33)
10  out of  (232, 33)
20  out of  (232, 33)
30  out of  (232, 33)
40  out of  (232, 33)
50  out of  (232, 33)
60  out of  (232, 33)
70  out of  (232, 33)
80  out of  (232, 33)
90  out of  (232, 33)
100  out of  (232, 33)
110  out of  (232, 33)
120  out of  (232, 33)
130  out of  (232, 33)
140  out of  (232, 33)
150  out of  (232, 33)
160  out of  (232, 33)
170  out of  (232, 33)
180  out of  (232, 33)
190  out of  (232, 33)
200  out of  (232, 33)
210  out of  (232, 33)
220  out of  (232, 33)
230  out of  (232, 33)
559
{'1627773', '201145', '203924', '202334', '1627758', '200768', '201599', '203906', '1628407', '203948', '203530', '203485', '1628414', '201588', '201196', '1627775', '1938', '201574', '1628425', '1627884', '1627816', '203992', '204025', '202711', '201583', '202734', '203112', '203200', '202704', '2863', '2216', '2756', '202389', '1626199', '2581', '1713', '1626168', '1626167', '1628393', '201935', '203087', '203918', '203894', '2772', '101179', '16261

In [ ]:
def not_really_scraping(player_ids):
    t0 = time()
    api_seasons = ["2017-18", "2016-17", "2015-16"]
    headers = ['player_id', 'min', 'fgm', 'fga', 'fg3m', 
    'fg3a', 'ftm', 'fta', 'oreb', 'dreb', 'reb', 'ast', 'tov','stl', 'blk', 'pts', 'season', 'player_vs_id', 'fp']
    not_input = {}
    for header in headers:
        not_input[header] = []
    count = 0
    responses = []
    
    for player_id in player_ids:
        count += 1
        
        for season in api_seasons:
            sleep(3)
            response = player.PlayerVsPlayer(2544,player_id, season=season).json
            print(response)
            responses.append(response)
            
            for header in headers:
                if header.upper() in response['resultSets'][0]['headers']:
                    idx = response['resultSets'][0]['headers'].index(header.upper())
                    not_input[header].append(str(response['resultSets'][0]['rowSet'][0][idx]))
                
            #add FP
            not_input['season'].append(season)
            not_input['player_vs_id'].append(player_id)
            not_input['fp'].append('meme')
            
        if not(count%10):
            print(count, " out of ", len(player_ids))
            print('Just completed: ' + str(count) + ' with an ETC of ' +  str( ((time() - t0)/(count)) * (len(player_ids) - (count) )))
            #assert(1==0)
    #pp.pprint(response['resultSets'][0])
    df = pd.DataFrame(data=not_input)
    df.to_pickle('./player_vs_table.pkl')
    pickle.dump(responses,open('./raw_player_matchup_responses.pkl','wb'))
    


In [25]:
def not_really_scraping_v2(player_ids):
    t0 = time()
    api_seasons = ["2017-18", "2016-17", "2015-16"]
    headers = ['player_id', 'min', 'fgm', 'fga', 'fg3m', 
    'fg3a', 'ftm', 'fta', 'oreb', 'dreb', 'reb', 'ast', 'tov','stl', 'blk', 'pts', 'season', 'player_vs_id']
    not_input = {}
    for header in headers:
        not_input[header] = []
    count = 0
    responses = pd.read_pickle('../data/raw_player_matchup_responses.pkl')
    responses.reverse()

    
    for player_id in player_ids:
        count += 1
        for season in api_seasons:
            response = responses.pop()
            played = False
            for header in headers:
                if header.upper() in response['resultSets'][1]['headers']:
                    idx = response['resultSets'][1]['headers'].index(header.upper())
                    if len(response['resultSets'][1]['rowSet']):
                        played = True
                        not_input[header].append(str(response['resultSets'][1]['rowSet'][0][idx]))
                    else:
                        break
            if played:
                not_input['season'].append(season)
                not_input['player_vs_id'].append(player_id)
            
        if not(count%10):
            print(count, " out of ", len(player_ids))
            print('Just completed: ' + str(count) + ' with an ETC of ' +  str( ((time() - t0)/(count)) * (len(player_ids) - (count) )))
    print(not_input)
    df = pd.DataFrame(data=not_input)
    df.to_pickle('./player_vs_table_2.pkl')
    
    

In [26]:
not_really_scraping_v2(player_ids)

10  out of  559
Just completed: 10 with an ETC of 20.558939409255984
20  out of  559
Just completed: 20 with an ETC of 10.146704196929932
30  out of  559
Just completed: 30 with an ETC of 6.682712912559509
40  out of  559
Just completed: 40 with an ETC of 4.943008428812027
50  out of  559
Just completed: 50 with an ETC of 3.8984244060516358
60  out of  559
Just completed: 60 with an ETC of 3.2093205412228905
70  out of  559
Just completed: 70 with an ETC of 2.709810999461583
80  out of  559
Just completed: 80 with an ETC of 2.3374496489763263
90  out of  559
Just completed: 90 with an ETC of 2.044700985484653
100  out of  559
Just completed: 100 with an ETC of 1.8078167724609375
110  out of  559
Just completed: 110 with an ETC of 1.6178126855330033
120  out of  559
Just completed: 120 with an ETC of 1.4553886791070303
130  out of  559
Just completed: 130 with an ETC of 1.3210262775421142
140  out of  559
Just completed: 140 with an ETC of 1.206871007169996
150  out of  559
Just complet